In [7]:
import os, sys
import json
import random
from collections import defaultdict
from itertools import chain

In [2]:
KORQUAD_PATH = '/home/ubuntu/workspace/kaist.ir/qa/data/korquad'
BOOKQA_PATH = '/home/ubuntu/workspace/kaist.ir/qa/data/bookqa'
NEWSQA_PATH = '/home/ubuntu/workspace/kaist.ir/qa/data/newsqa'

KORQUAD_TRAIN = 'KorQuAD_v1.0_train.json'
KORQUAD_TEST = 'KorQuAD_v1.0_dev.json'
BOOKQA_TRAIN = 'bookqa_train.json'
BOOKQA_TEST = 'bookqa_valid.json'
NEWS_ALL = 'ko_nia_normal_squad_all.json'

In [3]:
with open(os.path.join(KORQUAD_PATH, KORQUAD_TRAIN), 'r') as f:
    korquad_train = json.load(f)

with open(os.path.join(NEWSQA_PATH, NEWS_ALL), 'r') as f:
    news_all = json.load(f)

with open(os.path.join(BOOKQA_PATH, BOOKQA_TRAIN), 'r') as f:
    bookqa_train = json.load(f)

In [4]:
news_dict = defaultdict(list)
for data in news_all['data']:
    source = data['source']
    news_dict[source].append(data)
print(news_dict.keys())

dict_keys([5, 7, 4, 3, 2, 6, 9, 1, 8])


In [15]:
seeds = [10, 20, 30, 40, 50]
for seed in seeds:
    random.seed(seed)
    news_train = defaultdict(list)
    news_test = defaultdict(list)
    for k, v in news_dict.items():
        test_idx = random.choices(list(range(len(news_dict[k]))), k=len(news_dict[k])//10)
        for ix, paragraph in enumerate(v):
            if ix in test_idx:
                news_test[k].append(paragraph)
            else:
                news_train[k].append(paragraph)

    news_train_out = {"data": list(chain.from_iterable(news_train.values()))}
    news_test_out = {"data": list(chain.from_iterable(news_test.values()))}

    with open(f"../data/newsqa/news_train_all_{seed}.json", "w") as f:
        json.dump(news_train_out, f)
    with open(f"../data/newsqa/news_test_all_{seed}.json", "w") as f:
        json.dump(news_test_out, f)

42802

In [43]:
tmp_out = {"data": news_train[1]}
tmp_out.keys()

dict_keys(['data'])

In [51]:
with open("tmp_out.json" , encoding= "utf-8", mode="w") as f:
    json.dump(tmp_out, f)

In [22]:
with open("/home/ubuntu/workspace/kaist.ir/qa/data/newsqa/tmp_out.json", 'r') as f:
    test = json.load(f)
len(test['data'])

6618

In [110]:
korquad_train_tmp = {"data": korquad_train['data'][:20]}
korquad_test_tmp = {"data": korquad_train['data'][20:30]}
hub_train_tmp = {"data": news_train[1][:20]}
hub_test_tmp = {"data": news_train[1][20:30]}

with open("/home/ubuntu/workspace/kaist.ir/qa/data/tmp/korquad_train_tmp.json" , encoding= "utf-8", mode="w") as f:
    json.dump(korquad_train_tmp, f)

with open("/home/ubuntu/workspace/kaist.ir/qa/data/tmp/korquad_test_tmp.json" , encoding= "utf-8", mode="w") as f:
    json.dump(korquad_test_tmp, f)

with open("/home/ubuntu/workspace/kaist.ir/qa/data/tmp/hub_train_tmp.json" , encoding= "utf-8", mode="w") as f:
    json.dump(hub_train_tmp, f)

with open("/home/ubuntu/workspace/kaist.ir/qa/data/tmp/hub_test_tmp.json" , encoding= "utf-8", mode="w") as f:
    json.dump(hub_test_tmp, f)

# datasets.py Check

In [17]:
import os, sys
import inspect
from tqdm import tqdm

In [95]:
sys.path.append(os.path.dirname(os.getcwd()))

In [90]:
class SquadExample(object):
    """
    A single training/test example for the Squad dataset.
    For examples without an answer, the start and end position are -1.
    """

    def __init__(self,
                 qas_id,
                 question_text,
                 doc_tokens,
                 orig_answer_text=None,
                 start_position=None,
                 end_position=None,
                 is_impossible=None):
        self.qas_id = qas_id
        self.question_text = question_text
        self.doc_tokens = doc_tokens
        self.orig_answer_text = orig_answer_text
        self.start_position = start_position
        self.end_position = end_position
        self.is_impossible = is_impossible

    def __str__(self):
        return self.__repr__()

    def __repr__(self):
        s = ""
        s += "qas_id: %s" % (self.qas_id)
        s += ", question_text: %s" % (
            self.question_text)
        s += ", doc_tokens: [%s]" % (" ".join(self.doc_tokens))
        if len(self.orig_answer_text) != 0:
            s += ", orig_answer_text: %s" % (self.orig_answer_text)
        if self.start_position != -1:
            s += ", start_position: %d" % (self.start_position)
        if self.end_position != -1:
            s += ", end_position: %d" % (self.end_position)
        if self.is_impossible:
            s += ", is_impossible: %r" % (self.is_impossible)
        return s

class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self,
                 unique_id,
                 example_index,
                 doc_span_index,
                 tokens,
                 token_to_orig_map,
                 token_is_max_context,
                 input_ids,
                 input_mask,
                 segment_ids,
                 start_position=None,
                 end_position=None,
                 is_impossible=None,
                 pooled_feature=None,
                 cluster_idx=None):
        self.unique_id = unique_id
        self.example_index = example_index
        self.doc_span_index = doc_span_index
        self.tokens = tokens
        self.token_to_orig_map = token_to_orig_map
        self.token_is_max_context = token_is_max_context
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.start_position = start_position
        self.end_position = end_position
        self.is_impossible = is_impossible
        self.pooled_feature = pooled_feature
        self.cluster_idx = cluster_idx

def _whitespace_tokenize(text):
    """Runs basic whitespace cleaning and splitting on a piece of text."""
    text = text.strip()
    if not text:
        return []
    tokens = text.split()
    return tokens

def read_examples(input_file, is_training):
    """Read a SQuAD json file into a list of SquadExample."""
    with open(input_file, "r", encoding='utf-8') as reader:
        input_data = json.load(reader)["data"]

    def is_whitespace(c):
        if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
            return True
        return False

    examples = []
    for entry in tqdm(input_data):
        for paragraph in entry["paragraphs"]:
            paragraph_text = paragraph["context"]
            doc_tokens = []
            char_to_word_offset = []
            prev_is_whitespace = True
            for c in paragraph_text:
                if is_whitespace(c):
                    prev_is_whitespace = True
                else:
                    if prev_is_whitespace:
                        doc_tokens.append(c)
                    else:
                        doc_tokens[-1] += c
                    prev_is_whitespace = False
                char_to_word_offset.append(len(doc_tokens) - 1)
            #pdb.set_trace()
            for qa in paragraph["qas"]:
                qas_id = qa["id"]
                question_text = qa["question"]
                start_position = None
                end_position = None
                orig_answer_text = None
                is_impossible = False
                if is_training:
                    if (len(qa["answers"]) != 1) and (not is_impossible):
                        raise ValueError(
                            "For training, each question should have exactly 1 answer.")
                    if not is_impossible:
                        answer = qa["answers"][0]
                        orig_answer_text = answer["text"]
                        answer_offset = answer["answer_start"]
                        answer_length = len(orig_answer_text)
                        start_position = char_to_word_offset[answer_offset]
                        end_position = char_to_word_offset[answer_offset + answer_length - 1]
                        # Only add answers where the text can be exactly recovered from the
                        # document. If this CAN'T happen it's likely due to weird Unicode
                        # stuff so we will just skip the example.
                        #
                        # Note that this means for training mode, every example is NOT
                        # guaranteed to be preserved.
                        actual_text = " ".join(doc_tokens[start_position:(end_position + 1)])
                        cleaned_answer_text = " ".join(
                            _whitespace_tokenize(orig_answer_text))
                        if actual_text.find(cleaned_answer_text) == -1:
                            continue
                    else:
                        start_position = -1
                        end_position = -1
                        orig_answer_text = ""

                example = SquadExample(
                    qas_id=qas_id,
                    question_text=question_text,
                    doc_tokens=doc_tokens,
                    orig_answer_text=orig_answer_text,
                    start_position=start_position,
                    end_position=end_position,
                    is_impossible=is_impossible)
                examples.append(example)
    return examples

In [31]:
examples = read_examples('/home/ubuntu/workspace/kaist.ir/qa/data/korquad/KorQuAD_v1.0_dev.json', is_training=True)

100%|██████████| 140/140 [00:00<00:00, 485.75it/s]


In [40]:
examples[2].end_position

0

In [43]:
def is_whitespace(c):
    if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
        return True
    return False

In [92]:
is_training=True
input_data = korquad_train['data'][1:3]
examples = []
for entry in tqdm(input_data):
    for paragraph in entry["paragraphs"]:
        paragraph_text = paragraph["context"].replace('\u200b', '')
        doc_tokens = []
        prev_is_whitespace = True
        for c in paragraph_text:
            if is_whitespace(c):
                prev_is_whitespace = True
            else:
                if prev_is_whitespace:
                    doc_tokens.append(c)
                else:
                    doc_tokens[-1] += c
                prev_is_whitespace = False
        #pdb.set_trace()
        for qa in paragraph["qas"]:
            qas_id = qa["id"]
            question_text = qa["question"]
            answer = qa['answers'][0]
            is_impossible = qa['is_impossible'] if 'is_impossible' in qa.keys() else False
            
            if not is_impossible:
                orig_answer_text = answer['text']
                start_position = answer['answer_start']
                end_position = start_position + len(orig_answer_text)
            else:
                start_position = -1
                end_position = -1
                orig_answer_text = ""
        

            example = SquadExample(
                qas_id=qas_id,
                question_text=question_text,
                doc_tokens=doc_tokens,
                orig_answer_text=orig_answer_text,
                start_position=start_position,
                end_position=end_position,
                is_impossible=is_impossible)
            examples.append(example)

100%|██████████| 2/2 [00:00<00:00, 620.37it/s]


In [93]:
examples[0]

qas_id: 6561639-0-0, question_text: 모든 구강기관으로 여성의 성기를 애무하는 것을 뭐라고 하는가?, doc_tokens: [커닐링구스(커닐링거스, 쿤닐링구스, 영어: Cunnilingus)는 입술, 혀, 입 등의 모든 구강기관으로 여성의 성기를 애무하는 것을 말하며 구강성교(오럴섹스, 영어: Oral sex)라고도 한다. 보통 남성이 행하며 동성의 여성이 행하는 경우도 있다. 혀를 질에 넣거나 여성의 클리토리스, 외음부나 그 주변을 핥거나 빨아서 애무한다. 받는 사람이 다양한 성감을 느끼는 원인이 되며 특히 클리토리스의 감각이 매우 중요하다. 타액과 수성윤활제가 자주 사용되고 이것들은 부드럽게 매끄러운 자극을 가능하게 한다. 파트너의 반응에 귀를 기울이면서 손가락과 같은 다른 애무와 함께 몸 전체에 다양한 자극과 결합하여 양측이 폭넓은 즐거움을 나눌 수 있게 한다. 전희로서 하는 경우가 많지만, 오르가즘에의 도달 여부에 관계없이 커닐링구스 자체가 성행위이다.], orig_answer_text: 커닐링구스, start_position: 0, end_position: 5

In [79]:
korquad_train['data'][1]['paragraphs']

[{'qas': [{'answers': [{'text': '커닐링구스', 'answer_start': 0}],
    'id': '6561639-0-0',
    'question': '모든 구강기관으로 여성의 성기를 애무하는 것을 뭐라고 하는가?'},
   {'answers': [{'text': '클리토리스', 'answer_start': 218}],
    'id': '6561639-0-1',
    'question': '구강성교를 할 때 중요한 감각은 무엇인가?'},
   {'answers': [{'text': '커닐링구스', 'answer_start': 0}],
    'id': '6561642-0-0',
    'question': '구강기관으로 여성의 성기를 애무하는 행위를 뜻하는 용어는?'},
   {'answers': [{'text': '클리토리스', 'answer_start': 218}],
    'id': '6561642-0-1',
    'question': '구강성교시 특히 어느 부위의 감각이 중요한가?'},
   {'answers': [{'text': '구강성교', 'answer_start': 80}],
    'id': '6561653-0-0',
    'question': '여성의 성기를 애무하는 것을 말하는 커닐링구스를 다른 말로 무엇이라고 합니까?'},
   {'answers': [{'text': '클리토리스', 'answer_start': 218}],
    'id': '6561653-0-1',
    'question': '커닐링구스는 받는 사람이 다양한 성감을 느끼는 원인이 되며 무엇의 감각이 특히 중요합니까?'}],
  'context': '커닐링구스(커닐링거스, 쿤닐링구스, 영어: Cunnilingus)는 입술, 혀, 입 등의 모든 구강기관으로 여성의 성기를 애무하는 것을 말하며 구강성교(오럴섹스, 영어: Oral sex)라고도 한다. 보통 남성이 행하며 동성의 여성이 행하는 경우도 있다. 혀를 질에 넣거나 여성의 클리

In [70]:
bookqa_train['data'][0]['paragraphs'][0]

{'context': '또한, 정책 및 사업의 본격 추진을 위한 재원의 확보 방안을 마련해야 한다. 특별법의 하위 법령 또는 관련 법령의 제·개정을 통해 불이행 행위자에 강제 이행 유도를 위한 벌금 또는 ‘오염 원인자 부담’의 원칙에 따른 ‘해양 대기오염 부담금(안)’을 부과할 수 있다. 부담금 징수로 인한 재정 수익분을 항만의 대기환경 개선을 위한 특별회계에서 수용하도록 하고, 예산의 배분에 있어서도 사업의 시급성, 파급 효과 등을 고려하여 사업의 우선순위에 차등을 두어 지원하는 ‘선택과 집중’의 전략적 방법론을 적용해야 한다. 이와 더불어 유관 부처 및 기관 등 주요 정책 행위자들의 업무 역할과 기능, 나아가 의무 및 권한의 범위 등에 대한 명확한 법적 근거 역시 시급하게 마련되어야 한다. 각 부처 및 기관별로 사업을 추진하고 개별성과를 기계적으로 취합하는 현재의 방식보다는, 통합 관리 차원에서 유관 부처·기관 간 공동 연구개발 및 조사, 공공 데이터 차원의 정보 연계·공유 및 활용 등을 통해 정책 및 사업의 효율성을 높이고, 공동의 사회 문제 해결을 위한 시너지의 도출이 필요하다.',
 'qas': [{'question': '예산을 분배할 때 사업의 우선 순위를 정해서 차등 지원하는 방법을 뭐라고 하지',
   'id': 1063333,
   'is_impossible': False,
   'answers': [{'text': '선택과 집중', 'answer_start': 260}]},
  {'question': '사업의 우선 순위를 정해서 예산을 차등적으로 나눠주는 방법을 뭐라고 해',
   'id': 1063334,
   'is_impossible': False,
   'answers': [{'text': '선택과 집중', 'answer_start': 260}]},
  {'question': '사업의 우선 순위로 예산을 차등 분배하는 방법을 뭐라고 불러',
   'id': 1063335,
   'is_impossible': False,
   'answ

In [118]:
import torch

from transformers import AutoTokenizer
from customBertModel import CustomizedBertForQuestionAnswering

In [111]:
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
model = CustomizedBertForQuestionAnswering.from_pretrained("klue/bert-base")

Some weights of the model checkpoint at klue/bert-base were not used when initializing CustomizedBertForQuestionAnswering: ['cls.predictions.decoder.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing CustomizedBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomizedBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Cus

In [124]:
context = korquad_train['data'][0]['paragraphs'][0]['context']
question = korquad_train['data'][0]['paragraphs'][0]['qas'][0]['question']
answer = korquad_train['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['text']
start_position = korquad_train['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['answer_start']
end_position = start_position + len(answer)

encoding = tokenizer(
        context,
        question,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_attention_mask=True)

start_position = encoding.char_to_token(start_position)
end_position = encoding.char_to_token(end_position)

shift = 1
if start_position is None:
        start_position = 512
while end_position is None:
        end_position = encoding.char_to_token(end_position - shift)
        shift += 1

encoding.update({'start_positions': start_position, 'end_positions': end_position})
encoding = {k: torch.tensor(v) for k, v in encoding.items()}

input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']
start_positions = encoding['start_positions']
end_positions = encoding['end_positions']

outputs = model(input_ids=input_ids.reshape((1, len(input_ids))),
                    attention_mask=attention_mask.reshape((1, len(attention_mask))),
                    start_positions=start_positions,
                    end_positions=end_positions)

ValueError: Expected input batch_size (1) to match target batch_size (0).

In [127]:
print(model)

CustomizedBertForQuestionAnswering(
  (bert): CustomizedBertModel(
    (embeddings): CustomizedBertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((